In [1]:
## Note: please make sure navitron-crons is already installed in venv
## https://www.alfredo.motta.name/create-isolated-jupyter-ipython-kernels-with-pyenv-and-virtualenv/
from datetime import datetime, timedelta
from IPython.display import display, Markdown
import pandas as pd
import numpy as np
import plotnine

import navitron_crons.connections as connections
import navitron_crons.cli_core as cli_core

CONN = connections.MongoConnection(cli_core.CONFIG)
%matplotlib inline

/Users/jpurcell/source/NavitronEve/jvenv_navitron_mac/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [14]:
## Fetch map stats ##
with CONN as db_conn:
    raw_system_data = list(db_conn['navitron_system_stats'].find(
        {}, projection={'_id': False, 'metadata': False, 'write_recipt': False}))
    
system_jumps_df = pd.DataFrame(raw_system_data)
display(system_jumps_df.head(5))

,cron_datetime,npc_kills,pod_kills,ship_jumps,ship_kills,system_id
0,2017-10-21T00:06:16.264999,3,0,1,0,30003298
1,2017-10-21T00:06:16.264999,33,0,47,0,30002911
2,2017-10-21T00:06:16.264999,65,0,108,2,30002811
3,2017-10-21T00:06:16.264999,64,0,23,0,30000086
4,2017-10-21T00:06:16.264999,17,0,1,0,30003097


In [3]:
## Fetch server stats ##
with CONN as db_conn:
    raw_server_data = list(db_conn['navitron_server_status'].find(
        {}, projection={'_id': False, 'write_recipt': False}))
    
server_status_df = pd.DataFrame(raw_server_data)
display(server_status_df.head(5))

,cron_datetime,players,server_version,start_time
0,2017-10-23T19:08:27.555384,32108,1201367,2017-10-23T11:03:29Z
1,2017-10-23T20:08:41.266289,30403,1201367,2017-10-23T11:03:28Z
2,2017-10-23T21:08:24.454950,27227,1201367,2017-10-23T11:03:29Z
3,2017-10-23T22:08:22.193869,23802,1201367,2017-10-23T11:03:28Z
4,2017-10-23T23:08:25.111968,21580,1201367,2017-10-23T11:03:29Z


In [5]:
## Fetch map SDE ##
with CONN as db_conn:
    raw_map_sde = list(db_conn['navitron_sde_universe'].find(
        {}, projection={'_id': False, 'write_recipt': False, 'stargates': False, 'cron_datetime': False}))
    
map_sde_df = pd.DataFrame(raw_map_sde)
display(map_sde_df.head(5))

,constellation_id,constellation_name,region_id,region_name,security_class,security_status,solarsystem_name,star_id,system_id,x,y,z
0,20000256,Subhatoub,10000020,Tash-Murkon,B1,0.729366,Sinid,40110164,30001729,-2.101676e+17,5.862811e+16,-1.162771e+17
1,20000058,RO-AZT,10000004,UUA-F4,NaN,-0.000053,2E0P-2,40025036,30000403,1.043354e+17,9.060678e+16,3.121048e+17
2,20000374,Akimamur,10000030,Heimatar,E,0.404436,Wiskeber,40162485,30002554,-5.569225e+16,1.140439e+16,2.518558e+16
3,20000394,Stion,10000032,Sinq Laison,D1,0.453173,Stegette,40171501,30002696,-1.615165e+17,3.190137e+16,5.077709e+15
4,20000115,0YMH-Q,10000009,Insmother,F4,-0.576692,I-1QKL,40049757,30000788,1.182402e+17,3.912867e+16,-1.102126e+17


In [18]:
## Join all the things ##
system_jumps_df['hour'] = pd.to_datetime(system_jumps_df['cron_datetime']).dt.hour
system_jumps_df['date'] = pd.to_datetime(system_jumps_df['cron_datetime']).dt.date

server_status_df['hour'] = pd.to_datetime(server_status_df['cron_datetime']).dt.hour
server_status_df['date'] = pd.to_datetime(server_status_df['cron_datetime']).dt.date

## Join system_jumps vs server_status ##
full_df = pd.merge(
    system_jumps_df, server_status_df,
    on=['hour', 'date'],
    how='left'
)
full_df = full_df.drop(
    ['cron_datetime_y', 'start_time'],
    axis=1
)
full_df = full_df.dropna(axis=0, how='any')  # drop blank

## Join map_sde with all data
full_df = full_df.merge(
    map_sde_df,
    on='system_id',
    how='left'
)
full_df = full_df.drop(
    ['x', 'y', 'z', 'star_id', 'region_name', 'constellation_name'],
    axis=1
)

display(full_df)

,cron_datetime_x,npc_kills,pod_kills,ship_jumps,ship_kills,system_id,hour,date,players,server_version,constellation_id,region_id,security_class,security_status,solarsystem_name
0,2017-10-23T19:05:42.707410,18,0,13,0,30002997,19,2017-10-23,32108.0,1201367,20000439,10000036,B1,0.567127,Ehnoum
1,2017-10-23T19:05:42.707410,4,0,45,0,30002980,19,2017-10-23,32108.0,1201367,20000436,10000036,B2,0.335651,Sosan
2,2017-10-23T19:05:42.707410,2,0,14,0,30003847,19,2017-10-23,32108.0,1201367,20000562,10000048,D2,0.280538,Amoen
3,2017-10-23T19:05:42.707410,117,0,29,0,30004016,19,2017-10-23,32108.0,1201367,20000586,10000050,G,-0.152749,DG-L7S
4,2017-10-23T19:05:42.707410,162,0,84,0,30001426,19,2017-10-23,32108.0,1201367,20000209,10000016,C,0.725113,Isinokka
5,2017-10-23T19:05:42.707410,59,0,7,0,30000103,19,2017-10-23,32108.0,1201367,20000014,10000001,B1,0.525836,Serad
6,2017-10-23T19:05:42.707410,178,0,4,0,30004451,19,2017-10-23,32108.0,1201367,20000650,10000056,F4,-0.554116,E-PR0S
7,2017-10-23T19:05:42.707410,12,0,15,0,30000086,19,2017-10-23,32108.0,1201367,20000012,10000001,B1,0.658265,Sucha
8,2017-10-23T19:05:42.707410,4,0,13,0,30005084,19,2017-10-23,32108.0,1201367,20000744,10000065,B2,0.415573,Miroona
9,2017-10-23T19:05:42.707410,537,0,4,0,30004033,19,2017-10-23,32108.0,1201367,20000589,10000050,G6,-0.778777,ES-Q0W
